In [7]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import tensorflow_hub as hub

from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Dataset details 
image_height = 150
image_width = 150
number_of_channels = 3
number_of_classes = 6

In [3]:
#Hyper-Parameters
batch_size = 128
NUMBER_OF_EPOCHS = 10
LEARNING_RATE = 0.001

In [4]:
print("############-------- START --------############")
print("This code run on on Python version : ", sys.version)
# Print statement check tensorflow is running
print("The Tensorflow version used is : " + tf. __version__)

############-------- START --------############
This code run on on Python version :  3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]
The Tensorflow version used is : 2.10.0


In [5]:
# Load Image Data from local computer
# Use seg_train folder for training
train_set = tf.keras.preprocessing.image_dataset_from_directory(r"E:\Work\vs_code\Assessment_Oct22\SceneryDataset\seg_train",
color_mode= "rgb",
batch_size=batch_size,
image_size=(image_height, image_width),
shuffle=True,
seed=123)

# Use seg_test folder for hold-out validation
validation_set = tf.keras.preprocessing.image_dataset_from_directory(r"E:\Work\vs_code\Assessment_Oct22\SceneryDataset\seg_test",
color_mode= "rgb",
batch_size=batch_size,
image_size=(image_height, image_width),
shuffle=True,
seed=123)

print(train_set.class_names)
print(validation_set.class_names)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [ ]:
# Build the model
AUTOTUNE = tf.data.AUTOTUNE

base_model = tf.keras.applications.MobileNetV2(input_shape=(image_height, image_width, 3),
                                               include_top=False,
                                               weights='imagenet')

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


class_names = train_set.class_names
for image_batch, labels_batch in train_set:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# Data augmentation - options - flipped left-right as mirror,introduced perturbations in contrast and brightness and rotated by delta
image_augmentation = keras.Sequential(
    [       
        layers.RandomFlip(mode="horizontal",
                        input_shape=(image_height,
                                    image_width,
                                    3)),
        layers.RandomContrast(factor=0.1,),
        layers.RandomBrightness(factor=0.15),
        layers.RandomRotation(0.1)
    ]
)

train_set = train_set.cache().prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

# Model creation - this model includes dropout regularization
model = tf.keras.Sequential([
  image_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3,padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3,padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(number_of_classes)
])

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=NUMBER_OF_EPOCHS,
  callbacks=[lr_scheduler_callback]
)


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()